In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
!pip install transformers
!pip install datasets
!pip install wandb
!pip install -U datasets
!pip install objaverse
!pip install diffusers
!pip install trimesh
!pip install jaxtyping
!pip install pytorch-lightning
!pip install ijson
!pip install triton==3.2.0
!pip install wandb
!pip uninstall -y pillow
!pip install pillow==9.5.0 --no-cache-dir

In [ ]:
!pip install torch-cluster -f https://data.pyg.org/whl/torch-$(python -c "import torch; print(torch.__version__)").html
!apt-get update && apt-get install -y libaio-dev

In [1]:
import sys
!rm -rf AI-Game-Engine
!git clone https://github.com/Sakib323/AI-Game-Engine.git
sys.path.append('/kaggle/working/AI-Game-Engine')
from mmfreelm.models.hgrn_bit.mesh_dit import MeshDiT_models

Cloning into 'AI-Game-Engine'...
remote: Enumerating objects: 1249, done.
remote: Counting objects: 100% (254/254), done.
remote: Compressing objects: 100% (157/157), done.
remote: Total 1249 (delta 199), reused 149 (delta 97), pack-reused 995 (from 1)
Receiving objects: 100% (1249/1249), 463.43 MiB | 44.54 MiB/s, done.
Resolving deltas: 100% (846/846), done.
Updating files: 100% (242/242), done.


2025-11-29 17:58:06.268004: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764439086.294014    1996 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764439086.301370    1996 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [3]:
import requests
import gzip
import json
import objaverse
import gc
import os
import ijson
import objaverse
import pathlib
import shutil
import json
import torch
import torch.nn.functional as F
import traceback
import numpy as np
import trimesh
from tqdm import tqdm
import logging
import random
from torch.utils.data import Dataset
from transformers import AutoTokenizer, Trainer, TrainingArguments
from safetensors.torch import load_file as safetensors_load
import wandb
import shutil
from diffusers import DDIMScheduler
from mmfreelm.models.hgrn_bit.mesh_dit import MeshDiT_models
import signal
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
import psutil
import warnings
import zipfile
import shutil
from kaggle.api.kaggle_api_extended import KaggleApi
import os
import json
from kaggle.api.kaggle_api_extended import KaggleApi
import zipfile
import kagglehub
from contextlib import redirect_stdout
from io import StringIO
import signal
import time

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
os.environ["TOKENIZERS_PARALLELISM"] = "false"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float32
print(f"Using device: {device}, dtype: {dtype}")

print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained("Sakib323/MMfreeLM-370M")
tokenizer.pad_token = tokenizer.eos_token
print("Tokenizer loaded.")

Using device: cuda, dtype: torch.float32
Loading tokenizer...
Tokenizer loaded.


In [4]:
def train_model():
    import os, random, shutil
    import torch
    import torch.nn.functional as F
    from torch.utils.data import Dataset, Subset, RandomSampler, SequentialSampler, DataLoader
    from transformers import Trainer, TrainingArguments
    from diffusers import DDIMScheduler
    from tqdm.notebook import tqdm
    import wandb


    WANDB_TOKEN = "89b06c10468af620747b4bd340f72fa5d56f6849"
    try:
        wandb.login(key=WANDB_TOKEN)
        os.environ["WANDB_PROJECT"] = "mesh-dit-3d-generation"
        use_wandb = True
        print("Successfully logged in to W&B.")
    except Exception as e:
        print(f"Could not log in to W&B: {e}. Disabling W&B.")
        use_wandb = False

    print("\nCopying 4× dataset from Kaggle input → /tmp ...")
    kaggle_input_dir = "/kaggle/input/meshdit-all-data-5-pt"
    cache_dir = "/tmp/meshdit_cache"
    os.makedirs(cache_dir, exist_ok=True)
    file_names = ["all_data.pt","all_data_1.pt","all_data_2.pt","all_data_3.pt","all_data_4.pt"]
    local_paths = []
    for name in file_names:
        src = f"{kaggle_input_dir}/{name}"
        dst = f"{cache_dir}/{name}"
        if os.path.exists(dst):
            size_gb = os.path.getsize(dst) / (1024**3)
            print(f"Already cached: {name} ({size_gb:.2f} GB)")
        else:
            size_gb = os.path.getsize(src) / (1024**3)
            print(f"Copying {name} ({size_gb:.2f} GB) → /tmp ...")
            shutil.copy2(src, dst)
        local_paths.append(dst)
    print(f"All 4 files ready in {cache_dir}")


    class CustomDataCollator:
        def __call__(self, features):
            batch = {}
            batch['x'] = torch.stack([f['x'] for f in features])
            y_features = [f['y'] for f in features]
            batch['y'] = {key: torch.stack([d[key] for d in y_features]) for key in y_features[0]}
            return batch

    class MeshDiTTrainer(Trainer):
        def __init__(self, *args, noise_scheduler, **kwargs):
            super().__init__(*args, **kwargs)
            self.noise_scheduler = noise_scheduler

        def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
            x_start = inputs.get("x")
            model_kwargs = inputs.get("y")
            noise = torch.randn_like(x_start)
            timesteps = torch.randint(0, self.noise_scheduler.config.num_train_timesteps, (x_start.shape[0],), device=x_start.device).long()
            noisy_latents = self.noise_scheduler.add_noise(x_start, noise, timesteps)
            noise_pred = model(noisy_latents, timesteps, model_kwargs)
            loss = F.huber_loss(noise_pred, noise, delta=1.0)
            return (loss, {"loss": loss}) if return_outputs else loss

        def prediction_step(self, model, inputs, prediction_loss_only, ignore_keys=None):
            inputs = self._prepare_inputs(inputs)
            with torch.no_grad():
                loss = self.compute_loss(model, inputs)
            return (loss.detach(), None, None)


    class ConcatenatedMeshDataset(Dataset):
        def __init__(self, pt_paths):
            self.pt_paths = pt_paths
            self.cumsum = [0]
            self.lengths = []
            self.file_handles = []
            for path in pt_paths:
                print(f"Opening {os.path.basename(path)} with mmap (zero copy)...")
                handle = torch.load(path, map_location='cpu', mmap=True)
                data_list = handle['data']
                length = len(data_list)
                self.lengths.append(length)
                self.cumsum.append(self.cumsum[-1] + length)
                self.file_handles.append(data_list)
            print(f"Total samples: {sum(self.lengths):,} across {len(pt_paths)} files")

        def __len__(self):
            return self.cumsum[-1]

        def __getitem__(self, idx):
            for i, cum in enumerate(self.cumsum):
                if idx < cum:
                    file_idx = i - 1
                    local_idx = idx - self.cumsum[file_idx]
                    break
            data_list = self.file_handles[file_idx]
            sample = data_list[local_idx]
            return {'x': sample['x'], 'y': sample['y']}

    full_dataset = ConcatenatedMeshDataset(local_paths)
    total_len = len(full_dataset)
    eval_size = max(1, int(total_len * 0.10))
    train_size = total_len - eval_size
    train_subset = Subset(full_dataset, range(eval_size, total_len))
    eval_subset = Subset(full_dataset, range(eval_size))
    generator = torch.Generator().manual_seed(42)
    train_sampler = RandomSampler(train_subset, generator=generator)
    eval_sampler = SequentialSampler(eval_subset)
    print(f"Train: {train_size:,} | Eval: {eval_size:,}")

    data_collator = CustomDataCollator()
    noise_scheduler = DDIMScheduler(
        num_train_timesteps=1000,
        beta_schedule="scaled_linear",
        prediction_type="epsilon",
        clip_sample=False,
    )


    print("\n" + "="*6060)
    print("PHASE 1: Training from scratch (FAST!)")
    print("="*60)

    model_p1 = MeshDiT_models['MeshDiT-S'](
        input_tokens=2048,
        vocab_size=tokenizer.vocab_size,
        use_rope=True,
        use_ternary_rope=True,
        image_condition=False,
        full_precision=True,
        optimized_bitlinear=False,
    ).to(device, dtype=dtype)
    
    #state_dict_p1 = safetensors_load("/kaggle/input/meshdit-trained-model/model.safetensors", device="cpu")
    #model_p1.load_state_dict(state_dict_p1)
    #print("✅ Phase 1 weights loaded for fine-tuning")
    

    print(model_p1)

    training_args_p1 = TrainingArguments(
        output_dir="./phase3_ckpt",
        num_train_epochs=3,
        per_device_train_batch_size=8, 
        gradient_accumulation_steps=8,
        learning_rate=1e-4,
        lr_scheduler_type="cosine",  
        weight_decay=0.01,
        warmup_ratio=0.1,        
        logging_steps=50,
        eval_strategy="steps",
        eval_steps=500,
        save_strategy="steps",
        save_steps=500,
        save_total_limit=3,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        fp16=True,
        max_grad_norm=5.0,          
        dataloader_num_workers=4,
        remove_unused_columns=False,
        report_to="wandb" if use_wandb else "tensorboard",
        run_name="MeshDiT-S-Phase1-Final",
        dataloader_pin_memory=True,
        dataloader_prefetch_factor=4,
    )

    trainer_p1 = MeshDiTTrainer(
        model=model_p1,
        args=training_args_p1,
        train_dataset=train_subset,
        eval_dataset=eval_subset,
        data_collator=data_collator,
        noise_scheduler=noise_scheduler,
    )

    trainer_p1.train_dataloader = DataLoader(
        train_subset,
        batch_size=training_args_p1.per_device_train_batch_size,
        sampler=train_sampler,
        collate_fn=data_collator,
        num_workers=2,
        pin_memory=True,
        prefetch_factor=8,
        persistent_workers=True,
    )
    trainer_p1.eval_dataloader = DataLoader(
        eval_subset,
        batch_size=training_args_p1.per_device_train_batch_size,
        sampler=eval_sampler,
        collate_fn=data_collator,
        num_workers=2,
        pin_memory=True,
    )

    trainer_p1.train()
    trainer_p1.save_model("./mesh_dit_final")
    tokenizer.save_pretrained("./mesh_dit_final")

    print("\nTRAINING 100% COMPLETE!")

In [ ]:
if __name__ == "__main__":
    train_model()

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sakibahmed2018go to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Successfully logged in to W&B.

Copying 4× dataset from Kaggle input → /tmp ...
Already cached: all_data.pt (9.54 GB)
Already cached: all_data_1.pt (8.79 GB)
Already cached: all_data_2.pt (9.87 GB)
Already cached: all_data_3.pt (9.91 GB)
Already cached: all_data_4.pt (9.66 GB)
All 4 files ready in /tmp/meshdit_cache
Opening all_data.pt with mmap (zero copy)...
Opening all_data_1.pt with mmap (zero copy)...
Opening all_data_2.pt with mmap (zero copy)...
Opening all_data_3.pt with mmap (zero copy)...
Opening all_data_4.pt with mmap (zero copy)...
Total samples: 97,308 across 5 files
Train: 87,578 | Eval: 9,730

PHASE 1: Training from scratch (FAST!)
Initializing RotaryEmbedding with theta=10000.0 and ternary=True

[RotaryEmbedding] Initialized with: dim=64, max_pos=2048, base=10000.0, ternary=True

Initializing RotaryEmbedding with theta=10000.0 and ternary=True

[RotaryEmbedding] Initialized with: dim=64, max_pos=2048, base=10000.0, ternary=True

Initializing RotaryEmbedding with theta=

Step,Training Loss,Validation Loss
500,0.154400,0.154419
1000,0.157600,0.151122
1500,0.151600,0.148328
2000,0.146900,0.146269
2500,0.146300,0.146333


In [ ]:
import os
import json
import shlex
import subprocess
from datetime import datetime

KAGGLE_USERNAME = "sakibahmed2022"
KAGGLE_KEY = "d0ec7fe8091a3906c6995568c11e9e58" 
os.environ['KAGGLE_USERNAME'] = KAGGLE_USERNAME
os.environ['KAGGLE_KEY'] = KAGGLE_KEY

DESIRED_SLUG    = f"{KAGGLE_USERNAME}/meshdit-trained-model"
LOCAL_DIR       = "./mesh_dit_final"
TITLE           = "MeshDiT-S-Model-0.08x"
VERSION_MSG     = f"Upload final model – {datetime.utcnow().isoformat()} UTC"
PUBLIC          = False        

def write_metadata(folder: str, dataset_id: str):
    meta = {
        "title": TITLE,
        "id": dataset_id,
        "licenses": [{"name": "CC-BY-4.0"}],
        "description": f"{TITLE} – uploaded {datetime.utcnow().isoformat()} UTC"
    }
    path = os.path.join(folder, "dataset-metadata.json")
    with open(path, "w", encoding="utf-8") as f:
        json.dump(meta, f, indent=2)
    print(f"[meta] {path}")


def try_api(folder: str, dataset_id: str, msg: str):
    from kaggle.api.kaggle_api_extended import KaggleApi
    api = KaggleApi()
    api.authenticate()

    try:
        api.dataset_create_version(
            folder=folder,
            version_notes=msg,
            convert_to_csv=False,
            delete_old_versions=False,
        )
        print("[api] version created")
        return True, dataset_id
    except Exception as e:
        if "403" in str(e):
            print("[api] 403 after file upload → will use CLI")
            return False, dataset_id
        if "does not exist" in str(e).lower() or "404" in str(e):
            print("[api] dataset missing → creating new")
            api.dataset_create_new(folder=folder, public=PUBLIC, convert_to_csv=False)
            print("[api] new dataset created")
            return True, dataset_id
        print(f"[api] unexpected error: {e}")
        return False, None


def try_cli(folder: str, dataset_id: str, msg: str):
    # 1. version
    cmd = f"kaggle datasets version -p {shlex.quote(folder)} -m {shlex.quote(msg)}"
    print(f"[cli] version: {cmd}")
    p = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    if p.returncode == 0:
        print("[cli] version OK")
        return True, dataset_id

    cmd = f"kaggle datasets create -p {shlex.quote(folder)}"
    print(f"[cli] create: {cmd}")
    p = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    if p.returncode == 0:
        print("[cli] create OK")
        return True, dataset_id

    print("[cli] both commands failed")
    print("STDOUT:", p.stdout)
    print("STDERR:", p.stderr)
    return False, None

def upload():
    if not os.path.isdir(LOCAL_DIR):
        raise FileNotFoundError(LOCAL_DIR)

    files = [f for f in os.listdir(LOCAL_DIR) if os.path.isfile(os.path.join(LOCAL_DIR, f))]
    print("[main] files:", files)

    dataset_id = DESIRED_SLUG
    ts = datetime.utcnow().strftime("%Y%m%d%H%M%S")
    alt = f"{KAGGLE_USERNAME}/meshdit-trained-model-{ts}"

    try:
        from kaggle.api.kaggle_api_extended import KaggleApi
        KaggleApi().authenticate()
        KaggleApi().dataset_list_files(dataset_id)   # raises if missing
        print(f"[slug] '{dataset_id}' exists → using timestamped")
        dataset_id = alt
    except Exception:
        pass   # keep desired slug

    write_metadata(LOCAL_DIR, dataset_id)

    # 1. API
    print("[main] trying Python API …")
    ok, final_id = try_api(LOCAL_DIR, dataset_id, VERSION_MSG)
    if ok:
        url = f"https://www.kaggle.com/datasets/{final_id}"
        print(f"[SUCCESS] API → {url}")
        return url

    # 2. CLI
    print("[main] API failed → CLI fallback …")
    ok, final_id = try_cli(LOCAL_DIR, dataset_id, VERSION_MSG)
    if ok:
        url = f"https://www.kaggle.com/datasets/{final_id}"
        print(f"[SUCCESS] CLI → {url}")
        return url

    raise RuntimeError("Both API and CLI failed")

# --------------------------------------------------------------
# 7. RUN
# --------------------------------------------------------------
if __name__ == "__main__":
    final_url = upload()
    print("\n=== FINAL URL ===")
    print(final_url)
    print("\nDownload with:")
    print(f"kaggle datasets download -d {final_url.split('/')[-2]}/{final_url.split('/')[-1]}")